# Synth. Obs.: 3D Phantom - Reduction

We create synthetic observations for the Magritte model of the 3D Phantom snapshot that was created in the [this example](../1_post-processing/3_create_Phantom_3D.ipynb).

## Setup

Import the required functionalty.

In [1]:
from magrittetorch.model.model import Model
import magrittetorch.tools.plot as plot
import magrittetorch.algorithms.solvers as solvers
from astropy import units, constants
import os
import torch

from astropy import units              # Unit conversions

Define a working directory (you will have to change this). We assume here that the scripts of the [this example](../1_post-processing/3_create_Phantom_3D.ipynb) have already been executed and go back to that working directory.

In [2]:
wdir = "/lhome/thomasc/Magrittetorch-examples/Phantom_3D/"

Define file names.

In [3]:
model_file = os.path.join(wdir, 'model_Phantom_3D_red.hdf5')   # 3D Phantom Magrittetorch model

Load the Magritte model.

In [4]:
model = Model(model_file)
model.read()

Reading model from:  /lhome/thomasc/Magrittetorch-examples/Phantom_3D/model_Phantom_3D_red.hdf5
Reading Magrittetorch model


## Model the medium

Initialize the model

In [5]:
model.dataCollection.infer_data()

In this example we will work with the LTE level populations and **do not demand** statistical equilibrium.

In [6]:
# If you have a GPU, we can use it to speed up the computations
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Getting the full model on GPU might be a bit too much for its memory, so you might want to use your CPU instead for the computations
# device = torch.device("cpu")

# Compute level populations
# solvers.compute_level_populations(model, device=device, max_n_iterations=20)

## Make synthetic observations

Now we can make synthetic observations of the model.

In [7]:
# Get the line frequency corresponding to the CO 2-1 line
fcen = model.sources.lines.lineProducingSpecies[0].linedata.frequency.get(device)[1]
vpix = 1500   # velocity pixel size [m/s] 
nfreqs = 31   # number of frequency bins
dd   = vpix * (nfreqs-1)/2 / constants.c.value # frequency bin size [Hz]
fmin = fcen - fcen*dd
fmax = fcen + fcen*dd

image_freqs = torch.linspace(fmin, fmax, nfreqs, device=device, dtype=torch.float64)

# Compute image in the specified direction at the given frequencies
solvers.image_model(model, torch.Tensor([0,0,1]).type(torch.float64).to(device), image_freqs, device)

## Plot observations

Plot the resulting channel maps with the Magritte matplotlib back end.

In [8]:
plot.image_mpl(
    model,
    image_nr =  -1,
    zoom     = 1.3,
    npix_x   = 256,
    npix_y   = 256,
    x_unit   = units.au,
    v_unit   = units.km / units.s,
    method = 'nearest'
)

100%|████████████████████████████████████████████████████| 31/31 [00:16<00:00,  1.92it/s]


interactive(children=(IntSlider(value=15, description='v', max=30), Output()), _dom_classes=('widget-interact'…

<function magrittetorch.tools.plot.image_mpl.<locals>.<lambda>(v)>

(The plot is only interactive in a live notebook.)

Save the image cube in a fits file.

In [9]:
plot.save_fits(model)

Written file to: /lhome/thomasc/Magrittetorch-examples/Phantom_3D/images/image.fits


(Optional: To create your own plots) Overview of data stored in the Image object

In [10]:
import numpy as np
xdir = np.array(model.images[-1].image_direction_x.get())#directions of the x-and y-vectors of the image
ydir = np.array(model.images[-1].image_direction_y.get())
zdir = np.array(model.images[-1].image_direction_z.get())#this is direction in which we observe the object
print("image directions: ", xdir, ydir, zdir)
nfreqs = np.array(model.images[-1].nfreqs.get()) #number of frequency bins
freqs = np.array(model.images[-1].freqs.get_astropy()) #frequency bins [Hz]
print("# of frequencies: ", nfreqs, " frequencies :", freqs)
ImX = np.array(model.images[-1].imX.get_astropy())#X position in image [m]
ImY = np.array(model.images[-1].imY.get_astropy())#Y position in image [m]
I = np.array(model.images[-1].I.get_astropy())#Intensity at the corresponding ImX, ImY position [W/(m^2*Hz*sr)], at a given frequency bin
# print("Intensities :", I, " ImX:", ImX, "ImY:", ImY) #prints a lot of output

image directions:  [1. 0. 0.] [ 0.000000e+00  1.000000e+00 -6.123234e-17] [0.000000e+00 6.123234e-17 1.000000e+00]
# of frequencies:  31  frequencies : [2.30519701e+11 2.30520854e+11 2.30522008e+11 2.30523161e+11
 2.30524315e+11 2.30525468e+11 2.30526622e+11 2.30527775e+11
 2.30528929e+11 2.30530082e+11 2.30531236e+11 2.30532389e+11
 2.30533543e+11 2.30534696e+11 2.30535850e+11 2.30537003e+11
 2.30538156e+11 2.30539310e+11 2.30540463e+11 2.30541617e+11
 2.30542770e+11 2.30543924e+11 2.30545077e+11 2.30546231e+11
 2.30547384e+11 2.30548538e+11 2.30549691e+11 2.30550845e+11
 2.30551998e+11 2.30553152e+11 2.30554305e+11]
